In [2]:
Title
Introduction:
Preliminary exploratory data analysis:

Methods:

Expected outcomes and significance:


ERROR: Error in parse(text = x, srcfile = src): <text>:3:13: unexpected symbol
2: Introduction:
3: Preliminary exploratory
               ^
